In [18]:
import scipy.io as sio
import numpy as np
from matplotlib.colors import ListedColormap
from scipy.stats import pearsonr
import seaborn as sn
import mne
import matplotlib.pyplot as plt
%matplotlib qt
%gui qt

# File names for the baseline and signal files
str_FileName_R = 'Sujeto9_R_processed.fif'  # Name of the baseline file
str_FileName_MF = 'Sujeto9_MF_processed.fif'  # Name of the signal file
fileR = mne.read_epochs(str_FileName_R)  
fileMF = mne.read_epochs(str_FileName_MF)

# Get the data 
data_R = fileR.get_data()  # Shape: (n_epochs, n_channels, n_times)
data_MF = fileMF.get_data()
normalized_data = np.zeros_like(data_MF)

Reading c:\Users\afcad\Downloads\Sujeto9_R_processed.fif ...
    Found the data of interest:
        t =   -2500.00 ...    2500.00 ms
        0 CTF compensation matrices available


C:\Users\afcad\AppData\Local\Temp\ipykernel_24308\2762522924.py:14: RuntimeWarning: This filename (Sujeto9_R_processed.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  fileR = mne.read_epochs(str_FileName_R)


Not setting metadata
60 matching events found
No baseline correction applied
0 projection items activated
Reading c:\Users\afcad\Downloads\Sujeto9_MF_processed.fif ...
    Found the data of interest:
        t =   -2500.00 ...    2500.00 ms
        0 CTF compensation matrices available


C:\Users\afcad\AppData\Local\Temp\ipykernel_24308\2762522924.py:15: RuntimeWarning: This filename (Sujeto9_MF_processed.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  fileMF = mne.read_epochs(str_FileName_MF)


Not setting metadata
143 matching events found
No baseline correction applied
0 projection items activated


## PSD in two different ways 

In [19]:
###Plot signal and PSD

import numpy as np
import mne
import matplotlib.pyplot as plt

#fileR.plot(n_epochs=1, picks="eeg")
# fileMF.plot(n_epochs=1, picks="eeg", scalings="auto")

# Get the PSD data for both conditions
psd_R, freqs_R = fileR.compute_psd(method='welch', fmin=0.5, fmax=30).get_data(return_freqs=True)
psd_MF, freqs_MF = fileMF.compute_psd(method='welch', fmin=0.5, fmax=30).get_data(return_freqs=True)

# Number of channels
n_epochs_R, n_channels_R, _ = psd_R.shape
n_epochs_MF, n_channels_MF, _ = psd_MF.shape

# Plotting
fig, axs = plt.subplots(2, 1, figsize=(12, 8), sharex=True)

channel_names = fileR.ch_names
# Plot averaged PSD for each channel in the Resting condition
for ch in range(n_channels_R):  # Only first 4 channels for better visualization
    avg_psd_R = np.mean(psd_R[:, ch, :], axis=0)  # Average across epochs
    axs[0].plot(freqs_R, avg_psd_R, label=channel_names[ch])

# Plot averaged PSD for each channel in the MF condition
for ch in range(n_channels_MF):  # Only first 4 channels for better visualization
    avg_psd_MF = np.mean(psd_MF[:, ch, :], axis=0)  # Average across epochs
    axs[1].plot(freqs_MF, avg_psd_MF, label=channel_names[ch])

axs[0].set_title("PSD - Resting Condition")
axs[1].set_title("PSD - MF Condition")
axs[1].set_xlabel("Frequency (Hz)")
axs[0].set_ylabel("Power Spectral Density (dB)")
axs[1].set_ylabel("Power Spectral Density (dB)")

# Show legend with channel names
axs[0].legend(loc="upper right", fontsize=8)
axs[1].legend(loc="upper right", fontsize=8)

plt.tight_layout()
plt.show()

Effective window size : 2.000 (s)
Effective window size : 2.000 (s)


C:\Users\afcad\AppData\Local\Temp\ipykernel_24308\3063428839.py:42: UserWarning: Tight layout not applied. tight_layout cannot make Axes height small enough to accommodate all Axes decorations.
  plt.tight_layout()


In [3]:
##Testing fft

import numpy as np
import matplotlib.pyplot as plt
from scipy.fft import fft

# Assuming fileR.get_data() returns a 3D array (epochs, channels, time)
data = fileMF.get_data()  # 3D array of shape (epochs, channels, time)

# Extract the first channel across all epochs (shape: epochs, time)
data = data[:, 7, :]  

# Number of samples (time points) in each epoch
N = data.shape[1]  # Assuming the second dimension is the time (shape: epochs, time)
sampling_rate = 1024  # Sampling frequency in Hz

# Initialize a list to store the PSDs of each epoch
psds = []

# Iterate over each epoch
for epoch_data in data:
    # Perform the FFT for each epoch (only the first channel, which is already extracted)
    fft_result = fft(epoch_data)

    # Calculate the Power Spectral Density (PSD) as the squared magnitude of the FFT result
    psd = np.abs(fft_result) ** 2

    # Append the PSD to the list
    psds.append(psd)

# Convert the list of PSDs to a numpy array (shape: epochs, frequency bins)
psds = np.array(psds)

# Average the PSDs across all epochs
averaged_psd = np.mean(psds, axis=0)

# Calculate the corresponding frequency values
frequencies = np.fft.fftfreq(N, 1 / sampling_rate)

# Plot the averaged PSD (only positive frequencies)
plt.figure(figsize=(10, 6))
plt.plot(frequencies[:N // 2], averaged_psd[:N // 2])  # Plot the positive frequencies
plt.xlabel('Frequency (Hz)')
plt.ylabel('Average Power Spectral Density')
plt.title('Averaged PSD for First Channel Across Epochs')
plt.grid(True)
plt.xlim(0,50)
plt.show()

### Topomaps

In [20]:
import numpy as np
import mne
import matplotlib.pyplot as plt

# Compute PSDs using Welch's method
psd_R, freqs_R = fileR.compute_psd(method='welch', fmin=0.5, fmax=30).get_data(return_freqs=True)
psd_MF, freqs_MF = fileMF.compute_psd(method='welch', fmin=0.5, fmax=30).get_data(return_freqs=True)

# Average PSD across all epochs
avg_psd_R = np.mean(psd_R, axis=0)  # Shape: (n_channels, n_freqs)
avg_psd_MF = np.mean(psd_MF, axis=0)  # Shape: (n_channels, n_freqs)

# Define frequency bands
frequency_bands = {
    'Delta': (0.5, 4),
    'Theta': (4, 8),
    'Alpha': (8, 12),
    'Beta': (12, 18),
    'Fast Beta': (18, 30),
}

# Plot topomaps for resting state
info = fileR.info  # Use the baseline's info for channel locations
n_bands = len(frequency_bands)
fig, axes = plt.subplots(1, n_bands, figsize=(15, 5))  # Create a row of subplots

for ax, (band, (fmin, fmax)) in zip(axes, frequency_bands.items()):
    # Find indices for the current band
    band_idx = np.logical_and(freqs_R >= fmin, freqs_R <= fmax)
    
    # Average the power across the band
    band_power_R = np.mean(avg_psd_R[:, band_idx], axis=1)  # Mean over the frequency axis
    
    # Plot the topomap
    im, _ = mne.viz.plot_topomap(
        band_power_R, 
        info, 
        axes=ax, 
        cmap='jet', 
        show=False  # Disable immediate showing to allow subplot arrangement
    )
    ax.set_title(f'{band} Band (Resting)')
    cbar = plt.colorbar(im, ax=ax, shrink=0.6, orientation='horizontal', pad=0.1)
    cbar.set_label('Power')
    
plt.suptitle("Resting State Topomap", fontsize=16)
plt.tight_layout()
plt.show()

# Plot topomaps for mindfulness state
fig, axes = plt.subplots(1, n_bands, figsize=(15, 5))  # Create a row of subplots

for ax, (band, (fmin, fmax)) in zip(axes, frequency_bands.items()):
    # Find indices for the current band
    band_idx = np.logical_and(freqs_MF >= fmin, freqs_MF <= fmax)
    
    # Average the power across the band
    band_power_MF = np.mean(avg_psd_MF[:, band_idx], axis=1)  # Mean over the frequency axis
    
    # Plot the topomap
    im, _ = mne.viz.plot_topomap(
        band_power_MF, 
        info, 
        axes=ax, 
        cmap='jet', 
        show=False  # Disable immediate showing to allow subplot arrangement
    )
    ax.set_title(f'{band} Band (Mindfulness)')
    cbar = plt.colorbar(im, ax=ax, shrink=0.6, orientation='horizontal', pad=0.1)
    cbar.set_label('Power')

plt.suptitle("Mindfulness State Topomap", fontsize=16)
plt.tight_layout()
plt.show()

Effective window size : 2.000 (s)
Effective window size : 2.000 (s)


In [23]:
#Testing other function

import mne
import numpy as np
import matplotlib.pyplot as plt

# Define frequency range and FFT parameters
fmin, fmax = 0.5, 50  # Frequency range
n_fft = 256  # FFT length
tmax = 2.0  # Maximum time window for epochs
bands = [(1, 4, "Delta"), (4, 8, "Theta"), (8, 12, "Alpha"), (12, 18, "Beta slow"),(18, 30, "Beta fast"), (30, 50, "Gammma")]  # Frequency bands

# --- 1. PSD Plot for Raw Data ---
psd_R = fileR.compute_psd(fmin=fmin, fmax=fmax)
psd_R.plot(dB=True, ci="std")

psd_MF = fileMF.compute_psd(fmin=fmin, fmax=fmax)
psd_MF.plot(dB=True, ci="std")


####
# fig, axs = plt.subplots(3, 1)

# # Plot for psd_values_R in the first subplot (top left)
# psd_values_R, freqs = psd_R.get_data(return_freqs=True)
# for psd_value in psd_values_R:
#     axs[0].plot(freqs, 10 * np.log10(psd_value.mean(axis=0) / 1e-12))  # Plot the mean PSD across channels
# axs[0].set_xlabel("Frequency (Hz)")
# axs[0].set_ylabel("PSD (dB)")
# axs[0].set_title("PSD for psd_R")

# # Plot for psd_values_MF in the second subplot (top right)
# psd_values_MF, freqs = psd_MF.get_data(return_freqs=True)
# for psd_value in psd_values_MF:
#     axs[1].plot(freqs, 10 * np.log10(psd_value.mean(axis=0) / 1e-12))  # Plot the mean PSD across channels
# axs[1].set_xlabel("Frequency (Hz)")
# axs[1].set_ylabel("PSD (dB)")
# axs[1].set_title("PSD for psd_MF")

# # Calculate the difference (MF - R) and plot it in the third subplot (bottom left)
# for psd_value_R, psd_value_MF in zip(psd_values_R, psd_values_MF):
#     difference = 10 * np.log10(psd_value_MF.mean(axis=0) / 1e-12) - 10 * np.log10(psd_value_R.mean(axis=0) / 1e-12)
#     axs[2].plot(freqs, difference)  # Plot the difference in PSD (dB)
# axs[2].set_xlabel("Frequency (Hz)")
# axs[2].set_ylabel("Difference (dB)")
# axs[2].set_title("Difference (MF - R)")

# plt.tight_layout()
# plt.show()
####


# --- 2. Topomap of PSD for Epochs ---
# print(epochs_psd_R.get_data())
psd_R.plot_topomap(bands=bands, normalize=True, cmap='jet')
psd_MF.plot_topomap(bands=bands, normalize=True, cmap='jet')


# Get PSD data
psd_values_R, freqs = psd_R.get_data(return_freqs=True)
psd_values_MF, _ = psd_MF.get_data(return_freqs=True)

# Compute mean PSD across trials
mean_psd_R = np.mean(psd_values_R, axis=0)  # Shape: (n_channels, n_freqs)
mean_psd_MF = np.mean(psd_values_MF, axis=0)

# Compute the difference in (MF - R)
psd_diff = (mean_psd_MF / 1e-12) 

# Define frequency bands
bands = [(1, 4, "Delta"), (4, 8, "Theta"), (8, 12, "Alpha"),
         (12, 18, "Beta slow"), (18, 30, "Beta fast"), (30, 50, "Gamma")]

# Create topoplots for each band
fig, axes = plt.subplots(1, 6)  # Adjust grid for 6 plots


for ax, (fmin, fmax, name) in zip(axes.flatten(), bands):
    freq_idx = np.where((freqs >= fmin) & (freqs <= fmax))[0]
    psd_diff_band = psd_diff[:, freq_idx].mean(axis=1)  # Average over selected frequencies
    
    # Plot topomap
    im, _ = mne.viz.plot_topomap(psd_diff_band, psd_R.info, axes=ax, show=False, cmap="jet")
    ax.set_title(f"{name} [{fmin}-{fmax} Hz]")

# Add colorbar
fig.colorbar(im, ax=axes.ravel().tolist(), shrink=0.5)
plt.tight_layout()
plt.show()

    Using multitaper spectrum estimation with 7 DPSS windows
Plotting power spectral density (dB=True).
Averaging across epochs before plotting...
    Using multitaper spectrum estimation with 7 DPSS windows
Plotting power spectral density (dB=True).
Averaging across epochs before plotting...
Averaging across epochs before plotting...
converting legacy list-of-tuples input to a dict for the `bands` parameter
Averaging across epochs before plotting...
converting legacy list-of-tuples input to a dict for the `bands` parameter


C:\Users\afcad\AppData\Local\Temp\ipykernel_24308\3684406836.py:88: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()


In [6]:
import numpy as np
import mne
import matplotlib.pyplot as plt

# Compute PSDs using Welch's method
psd_R, freqs_R = fileR.compute_psd(method='multitaper', fmin=0.5, fmax=50).get_data(return_freqs=True)
psd_MF, freqs_MF = fileMF.compute_psd(method='multitaper', fmin=0.5, fmax=50).get_data(return_freqs=True)

# Average PSD across all windows (mean over epochs axis, axis=0)
avg_psd_R = np.mean(psd_R, axis=0)  # Shape: (n_channels, n_freqs)
avg_psd_MF = np.mean(psd_MF, axis=0)  # Shape: (n_channels, n_freqs)

# Calculate the difference
difference_psd = (avg_psd_MF - avg_psd_R)/1e-12  # Shape: (n_channels, n_freqs)

# Define frequency bands
frequency_bands = {
    'Delta': (0.5, 4),
    'Theta': (4, 8),
    'Alpha': (8, 12),
    'Beta': (12, 18),
    'Fast Beta': (18, 30),
    'Gamma': (30, 50)
}

# Calculate global min and max across all bands
global_min = np.inf
global_max = -np.inf

for fmin, fmax in frequency_bands.values():
    band_idx = np.logical_and(freqs_R >= fmin, freqs_R <= fmax)
    band_power_difference = np.mean(difference_psd[:, band_idx], axis=1)  # Mean over frequencies
    global_min = min(global_min, band_power_difference.min())
    global_max = max(global_max, band_power_difference.max())

# Extract band-specific differences and prepare the figure
info = fileR.info  # Use the baseline's info for channel locations
n_bands = len(frequency_bands)
fig, axes = plt.subplots(1, n_bands, figsize=(15, 5))  # Create a row of subplots

for ax, (band, (fmin, fmax)) in zip(axes, frequency_bands.items()):
    # Find indices for the current band
    band_idx = np.logical_and(freqs_R >= fmin, freqs_R <= fmax)
    
    # Average the difference across the band
    band_power_difference = np.mean(difference_psd[:, band_idx], axis=1)  # Mean over the frequency axis
    
    # Plot the topomap with consistent vlim
    im, _ = mne.viz.plot_topomap(
        band_power_difference, 
        info, 
        axes=ax, 
        cmap='jet', 
        show=False,  # Disable immediate showing to allow subplot arrangement
        names=fileR.ch_names,
    #   vlim=(global_min, global_max)  # Use global min and max
    )
    ax.set_title(f"{band} Band")
    # Add a colorbar for the subplot
    cbar = plt.colorbar(im, ax=ax, shrink=0.6, orientation='horizontal', pad=0.1)
    cbar.set_label('Power Difference')

# Finalize and display the figure
plt.suptitle("Mindfulness Difference", fontsize=16)
plt.tight_layout()
plt.show()

    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows


In [7]:
# import numpy as np
# import mne
# import matplotlib.pyplot as plt

# # Compute PSDs using Welch's method
# psd_R, freqs_R = fileR.compute_psd(method='welch', fmin=0.5, fmax=30).get_data(return_freqs=True)
# psd_MF, freqs_MF = fileMF.compute_psd(method='welch', fmin=0.5, fmax=30).get_data(return_freqs=True)

# # Normalize PSDs (relative power for each channel)
# total_power_R = np.sum(psd_R, axis=2, keepdims=True)  # Total power across frequencies per channel for R
# total_power_MF = np.sum(psd_MF, axis=2, keepdims=True)  # Total power across frequencies per channel for MF
# relative_psd_R = psd_R / total_power_R  # Shape: (n_epochs, n_channels, n_freqs)
# relative_psd_MF = psd_MF / total_power_MF

# # Average relative PSDs across epochs
# avg_relative_psd_R = np.mean(relative_psd_R, axis=0)  # Shape: (n_channels, n_freqs)
# avg_relative_psd_MF = np.mean(relative_psd_MF, axis=0)  # Shape: (n_channels, n_freqs)

# # Compute the relative power difference
# relative_power_difference = avg_relative_psd_MF - avg_relative_psd_R  # Shape: (n_channels, n_freqs)

# # Define frequency bands
# frequency_bands = {
#     'Delta': (0.5, 4),
#     'Theta': (4, 8),
#     'Alpha': (8, 12),
#     'Beta': (12, 18),
#     'FastBeta': (18, 30),
# }

# # Prepare the figure for topomaps
# info = fileR.info  # Use the baseline's info for channel locations
# n_bands = len(frequency_bands)
# fig, axes = plt.subplots(1, n_bands, figsize=(15, 5))  # Create a row of subplots

# for ax, (band, (fmin, fmax)) in zip(axes, frequency_bands.items()):
#     # Find indices for the current band
#     band_idx = np.logical_and(freqs_R >= fmin, freqs_R <= fmax)
    
#     # Average the relative power difference across the band
#     band_relative_power_diff = np.mean(relative_power_difference[:, band_idx], axis=1)  # Mean over frequencies
   
#     # Plot the topomap with a colorbar
#     im, _ = mne.viz.plot_topomap(
#         band_relative_power_diff, 
#         info, 
#         axes=ax, 
#         cmap='jet', 
#         show=False  # Disable immediate showing to allow subplot arrangement
#     )
#     ax.set_title(f'{band} Band')
    
#     # Add a colorbar for the subplot
#     cbar = plt.colorbar(im, ax=ax, shrink=0.6, orientation='horizontal', pad=0.1)
#     cbar.set_label('Relative Power Difference')

# # Finalize and display the figure
# plt.tight_layout()
# plt.show()

## Resting, Mind, DiffNormalized

In [8]:
import numpy as np
import mne
import matplotlib.pyplot as plt

# Compute PSDs using Welch's method
psd_R, freqs_R = fileR.compute_psd(method='welch', fmin=0.5, fmax=50).get_data(return_freqs=True)
psd_MF, freqs_MF = fileMF.compute_psd(method='welch', fmin=0.5, fmax=50).get_data(return_freqs=True)

# Normalize PSDs (relative power for each channel)
total_power_R = np.sum(psd_R, axis=2, keepdims=True)  # Total power across frequencies per channel for R
total_power_MF = np.sum(psd_MF, axis=2, keepdims=True)  # Total power across frequencies per channel for MF
relative_psd_R = psd_R / total_power_R  # Shape: (n_epochs, n_channels, n_freqs)
relative_psd_MF = psd_MF / total_power_MF

# Average relative PSDs across epochs
avg_relative_psd_R = np.mean(relative_psd_R, axis=0)   # Shape: (n_channels, n_freqs)
avg_relative_psd_MF = np.mean(relative_psd_MF, axis=0)   # Shape: (n_channels, n_freqs)

# Compute the relative power difference
relative_power_difference = avg_relative_psd_MF - avg_relative_psd_R  # Shape: (n_channels, n_freqs)

# Define frequency bands
frequency_bands = {
    'Delta': (0.5, 4),
    'Theta': (4, 8),
    'Alpha': (8, 12),
    'Beta': (12, 18),
    'FastBeta': (18, 30),
    'Gamma': (30, 50)
}

# Prepare the figure for topomaps
info = fileR.info  # Use the baseline's info for channel locations
n_bands = len(frequency_bands)
fig, axes = plt.subplots(3, n_bands, figsize=(11, 12),gridspec_kw={'wspace': 0.0001, 'hspace': 0.0001})  # Adjust spacing  # Create a grid of subplots

for row, data, title in zip(
    range(3),
    [avg_relative_psd_R, avg_relative_psd_MF, relative_power_difference],
    ['Resting', 'Mindfulness', 'Normalized Difference']
):
    for ax, (band, (fmin, fmax)) in zip(axes[row], frequency_bands.items()):
        # Find indices for the current band
        band_idx = np.logical_and(freqs_R >= fmin, freqs_R <= fmax)
        
        # Average the data across the band
        band_data = np.mean(data[:, band_idx], axis=1)  # Mean over frequencies
        
        # Plot the topomap with a colorbar
        im, _ = mne.viz.plot_topomap(
            band_data, 
            info, 
            axes=ax, 
            cmap='jet', 
            show=False,  # Disable immediate showing to allow subplot arrangement
        )
        ax.set_title(f'{band} Band' if row == 0 else '')
        
        # Add a colorbar for the subplot
        cbar = plt.colorbar(im, ax=ax, shrink=0.6, orientation='horizontal', pad=0.1)
        if row == 2:  # Label the colorbar only for the last row
            cbar.set_label(title)

# Add row titles
for ax, title in zip(axes[:, 0], ['Resting', 'Mindfulness', 'Normalized Difference']):
    ax.annotate(
        title, 
        xy=(0, 0.5), 
        xytext=(-ax.yaxis.labelpad - 5, 0),
        xycoords=ax.yaxis.label, 
        textcoords='offset points',
        size='large', ha='right', va='center', rotation=90
    )

# Finalize and display the figure
plt.tight_layout()
plt.show()

Effective window size : 2.000 (s)
Effective window size : 2.000 (s)


C:\Users\afcad\AppData\Local\Temp\ipykernel_24308\1618039706.py:76: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()


## For Several files (relative)

In [17]:
def topoplotsAverage(file_pairs, fmin=0.5, fmax=50):
    """
    Compute and plot the average PSDs and their differences for multiple pairs of EEG files,
    ensuring each row has the same color scale while different rows have different scales.

    Parameters:
    - file_pairs: List of tuples [(fileR1, fileMF1), (fileR2, fileMF2), ...]
                  Each tuple contains a resting state file and a mindfulness state file.
    - fmin: Minimum frequency for PSD calculation (default: 0.5 Hz).
    - fmax: Maximum frequency for PSD calculation (default: 30 Hz).

    Returns:
    - None (plots the results).
    """
    avg_psd_R_all = []
    avg_psd_MF_all = []

    for str_FileName_R, str_FileName_MF in file_pairs:
        fileR = mne.read_epochs(str_FileName_R)  
        fileMF = mne.read_epochs(str_FileName_MF)
        
        # Compute PSDs using Welch's method
        psd_R, freqs_R = fileR.compute_psd(method='welch', fmin=fmin, fmax=fmax).get_data(return_freqs=True)
        psd_MF, freqs_MF = fileMF.compute_psd(method='welch', fmin=fmin, fmax=fmax).get_data(return_freqs=True)

        # Normalize PSDs (relative power for each channel)
        total_power_R = np.sum(psd_R, axis=2, keepdims=True)
        total_power_MF = np.sum(psd_MF, axis=2, keepdims=True)
        relative_psd_R = psd_R / 1e-12
        relative_psd_MF = psd_MF / 1e-12

        # Average relative PSDs across epochs
        avg_relative_psd_R = np.mean(relative_psd_R, axis=0)
        avg_relative_psd_MF = np.mean(relative_psd_MF, axis=0)

        avg_psd_R_all.append(avg_relative_psd_R)
        avg_psd_MF_all.append(avg_relative_psd_MF)

    # Compute grand averages across all pairs
    avg_psd_R_all = np.mean(avg_psd_R_all, axis=0)
    avg_psd_MF_all = np.mean(avg_psd_MF_all, axis=0)

    # Compute the relative power difference
    relative_power_difference = avg_psd_MF_all - avg_psd_R_all

    # Define frequency bands
    frequency_bands = {
        'Delta': (0.5, 4),
        'Theta': (4, 8),
        'Alpha': (8, 12),
        'Beta': (12, 18),
        'FastBeta': (18, 30),
        'Gamma': (30, 50)
    }

    # Prepare the figure for topomaps
    info = mne.read_epochs(file_pairs[0][0]).info  # Use the info from the first resting state file
    n_bands = len(frequency_bands)
    fig, axes = plt.subplots(3, n_bands, figsize=(11, 12), gridspec_kw={'wspace': 0.001, 'hspace': 0.001})

    for row, data, title in zip(
        range(3),
        [avg_psd_R_all, avg_psd_MF_all, relative_power_difference],
        ['Resting', 'Mindfulness', 'Normalized Difference']
    ):
        # Initialize lists to store the min and max values across the row
        row_min = []
        row_max = []
        
        for ax, (band, (fmin, fmax)) in zip(axes[row], frequency_bands.items()):
            # Find indices for the current band
            band_idx = np.logical_and(freqs_R >= fmin, freqs_R <= fmax)

            # Average the data across the band
            band_data = np.mean(data[:, band_idx], axis=1)

            # Find the min and max values for the current band
            band_min = band_data.min()
            band_max = band_data.max()

            # Store the min and max for the row
            row_min.append(band_min)
            row_max.append(band_max)

            # Plot the topomap without setting vlim yet
            im, _ = mne.viz.plot_topomap(
                band_data,
                info,
                axes=ax,
                cmap='jet',
                show=False,
            )
            ax.set_title(f'{band} Band' if row == 0 else '')

            # Add a colorbar for the subplot
            cbar = plt.colorbar(im, ax=ax, shrink=0.6, orientation='horizontal', pad=0.1)
            if row == 2:
                cbar.set_label(title)

        # After collecting all min and max values for the row, set the vlim for each subplot
        row_vmin = min(row_min)
        row_vmax = max(row_max)

        # Apply the same vlim for all subplots in the current row
        for ax in axes[row]:
            im = ax.images[0]
            im.set_clim(vmin=row_vmin, vmax=row_vmax) #set the colorbar

    # Add row titles
    for ax, title in zip(axes[:, 0], ['Resting', 'Mindfulness', 'Normalized Difference']):
        ax.annotate(
            title,
            xy=(0, 0.5),
            xytext=(-ax.yaxis.labelpad - 5, 0),
            xycoords=ax.yaxis.label,
            textcoords='offset points',
            size='large', ha='right', va='center', rotation=90
        )

    # Finalize and display the figure
    plt.tight_layout()
    plt.show()

file_pairs = [
    ('Sujeto6_R_processed.fif', 'Sujeto6_MF_processed.fif'),
    ('Sujeto9_R_processed.fif', 'Sujeto9_MF_processed.fif'),
    ('Sujeto11_R_processed.fif', 'Sujeto11_MF_processed.fif'),
    ('Sujeto14_R_processed.fif', 'Sujeto14_MF_processed.fif'),
    ('Sujeto16_R_processed.fif', 'Sujeto16_MF_processed.fif'),
    ('Sujeto18_R_processed.fif', 'Sujeto18_MF_processed.fif'),
    ('Sujeto19_R_processed.fif', 'Sujeto19_MF_processed.fif'),
    ('Sujeto20_R_processed.fif', 'Sujeto20_MF_processed.fif'),
    ('Sujeto21_R_processed.fif', 'Sujeto21_MF_processed.fif'),
    ('Sujeto22_R_processed.fif', 'Sujeto22_MF_processed.fif')
]

topoplotsAverage(file_pairs)

Reading c:\Users\afcad\Downloads\Sujeto6_R_processed.fif ...
    Found the data of interest:
        t =   -2500.00 ...    2500.00 ms
        0 CTF compensation matrices available


C:\Users\afcad\AppData\Local\Temp\ipykernel_24308\3921912933.py:19: RuntimeWarning: This filename (Sujeto6_R_processed.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  fileR = mne.read_epochs(str_FileName_R)


Not setting metadata
72 matching events found
No baseline correction applied
0 projection items activated
Reading c:\Users\afcad\Downloads\Sujeto6_MF_processed.fif ...
    Found the data of interest:
        t =   -2500.00 ...    2500.00 ms
        0 CTF compensation matrices available


C:\Users\afcad\AppData\Local\Temp\ipykernel_24308\3921912933.py:20: RuntimeWarning: This filename (Sujeto6_MF_processed.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  fileMF = mne.read_epochs(str_FileName_MF)


Not setting metadata
107 matching events found
No baseline correction applied
0 projection items activated
Effective window size : 2.000 (s)
Effective window size : 2.000 (s)
Reading c:\Users\afcad\Downloads\Sujeto9_R_processed.fif ...
    Found the data of interest:
        t =   -2500.00 ...    2500.00 ms
        0 CTF compensation matrices available


...

C:\Users\afcad\AppData\Local\Temp\ipykernel_24308\3921912933.py:20: RuntimeWarning: This filename (Sujeto22_MF_processed.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  fileMF = mne.read_epochs(str_FileName_MF)


        t =   -2500.00 ...    2500.00 ms
        0 CTF compensation matrices available
Not setting metadata
138 matching events found
No baseline correction applied
0 projection items activated
Effective window size : 2.000 (s)
Effective window size : 2.000 (s)
Reading c:\Users\afcad\Downloads\Sujeto6_R_processed.fif ...
    Found the data of interest:
        t =   -2500.00 ...    2500.00 ms
        0 CTF compensation matrices available


C:\Users\afcad\AppData\Local\Temp\ipykernel_24308\3921912933.py:57: RuntimeWarning: This filename (Sujeto6_R_processed.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  info = mne.read_epochs(file_pairs[0][0]).info  # Use the info from the first resting state file


Not setting metadata
72 matching events found
No baseline correction applied
0 projection items activated


C:\Users\afcad\AppData\Local\Temp\ipykernel_24308\3921912933.py:121: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()


## Flattenizado

In [10]:
import mne
import numpy as np
import matplotlib.pyplot as plt

# Extract data
dataR = fileR.get_data()  # Shape: (n_epochs, n_channels, n_times)
dataMF = fileMF.get_data()  # Shape: (n_epochs, n_channels, n_times)

# Sampling frequency
sfreq = fileR.info['sfreq']  # Sampling frequency (Hz)
n_times_to_keep = int(dataR.shape[2] - 0.5 * sfreq)  # Time points to keep

# Trim the last 0.5 seconds
dataR_trimmed = dataR[:, :, :n_times_to_keep]
dataMF_trimmed = dataMF[:, :, :n_times_to_keep]

# print(dataR[:, 1, :])

# Compute mean and standard deviation of the trimmed resting signal
mean_R = np.mean(dataR_trimmed, axis=(0, 2))  # Mean across epochs and time
std_R = np.std(dataR_trimmed, axis=(0, 2))    # Std across epochs and time

# Concatenate both signals along epochs
data_combined = np.concatenate([dataR_trimmed, dataMF_trimmed], axis=0)

# Normalize the concatenated signal
#data_normalized = (data_combined)
data_normalized = (data_combined - mean_R[None, :, None]) / std_R[None, :, None] #Z score

# Concatenate all epochs along the time axis for each channel
data_normalized_ch = np.hstack(data_normalized)

# print(np.shape(data_normalized_ch))

# Assuming data_normalized_ch has shape (n_channels, total_time)
n_channels, total_time = data_normalized_ch.shape
time = np.linspace(0, total_time / sfreq, total_time)  # Generate time axis based on sampling frequency
electrode_names = fileR.ch_names  # List of electrode names

concat_point = dataR_trimmed.shape[0] * dataR_trimmed.shape[2] / sfreq  # Total time of dataR_trimmed
# Create the heatmap
plt.figure(figsize=(12, 6))
plt.imshow(data_normalized_ch, aspect='auto', cmap='viridis', 
           extent=[time[0], time[-1], n_channels, 0],vmin=-1,vmax=3)  # - 1e-5 , 1e-5

# Add a vertical line at the concatenation point
plt.axvline(x=concat_point, color='black', linestyle='--', label='Concatenation Point')

# Add labels and colorbar
plt.colorbar(label='Z-score')
plt.xlabel('Time (s)')
plt.ylabel('Electrodes')
plt.title('Heatmap of Normalized Signal Amplitudes')
plt.yticks(ticks=np.arange(len(electrode_names)), labels=electrode_names)
plt.show()

In [11]:
# Slice the data to include only the first 5 electrodes
data_subset = data_normalized_ch[:5, :]  # Shape: (5, total_time)

# Create a figure with subplots for heatmap and signals
fig, axes = plt.subplots(6, 1, figsize=(12, 12), gridspec_kw={'height_ratios': [5, 1, 1, 1, 1, 1]}, sharex=True)

# Plot the heatmap in the first subplot
im = axes[0].imshow(data_subset, aspect='auto', cmap='viridis',
                    extent=[time[0], time[-1], 5, 0],vmin=-1,vmax=3)
# Add a vertical line at the concatenation point
axes[0].axvline(x=concat_point, color='black', linestyle='--', label='Concatenation Point')

axes[0].set_ylabel('Electrodes')
axes[0].set_title('Heatmap of Normalized Signal Amplitudes (First 5 Electrodes)')
axes[0].yaxis.set_ticks(range(5))
axes[0].set_yticklabels([f'Electrode {i+1}' for i in range(5)])
plt.colorbar(im, ax=axes[0], orientation='horizontal', fraction=0.05, pad=0.1, label='Amplitude (normalized)')

# Plot signals in the next 5 subplots (one for each electrode)
for i in range(5):
    axes[i + 1].plot(time, data_subset[i], label=f'Electrode {i+1}')
    axes[i + 1].set_ylabel('Amplitude')
    axes[i + 1].legend(loc='upper right')

# Set the x-axis label for the last subplot
axes[-1].set_xlabel('Time (s)')

plt.tight_layout()
plt.show()


## For several Files signal vs time

In [12]:
import mne
import numpy as np
import matplotlib.pyplot as plt

def process_and_visualize_eeg(file_R, file_MF):
    """
    Process two EEG files (resting and MF signals), normalize them, and create a heatmap.
    
    Parameters:
        file_R (str): Filepath to the resting signal (.fif).
        file_MF (str): Filepath to the MF signal (.fif).
        
    Returns:
        np.ndarray: Normalized concatenated matrix of shape (n_channels, total_time).
    """
    # Load files
    fileR = mne.read_epochs(file_R)  
    fileMF = mne.read_epochs(file_MF)

    # Extract data
    dataR = fileR.get_data()  # Shape: (n_epochs, n_channels, n_times)
    dataMF = fileMF.get_data()  # Shape: (n_epochs, n_channels, n_times)

    # Sampling frequency
    sfreq = fileR.info['sfreq']  # Sampling frequency (Hz)
    n_times_to_keep = int(dataR.shape[2] - 0.5 * sfreq)  # Time points to keep

    # Trim the last 0.5 seconds
    dataR_trimmed = dataR[:, :, :n_times_to_keep]
    dataMF_trimmed = dataMF[:, :, :n_times_to_keep]

    # Compute mean and standard deviation of the trimmed resting signal
    mean_R = np.mean(dataR_trimmed, axis=(0, 2))  # Mean across epochs and time
    std_R = np.std(dataR_trimmed, axis=(0, 2))    # Std across epochs and time

    # Concatenate both signals along epochs
    data_combined = np.concatenate([dataR_trimmed, dataMF_trimmed], axis=0)

    # Normalize the concatenated signal
    data_normalized = (data_combined - mean_R[None, :, None]) / std_R[None, :, None]

    # Concatenate all epochs along the time axis for each channel
    data_normalized_ch = np.hstack(data_normalized)

    # Visualization
    n_channels, total_time = data_normalized_ch.shape
    time = np.linspace(0, total_time / sfreq, total_time)  # Generate time axis based on sampling frequency
    electrode_names = fileR.ch_names  # List of electrode names

    # Compute concatenation point in seconds
    concat_point = dataR_trimmed.shape[0] * dataR_trimmed.shape[2] / sfreq  # Total time of dataR_trimmed

    # # Create the heatmap
    # plt.figure(figsize=(12, 6))
    # plt.imshow(data_normalized_ch, aspect='auto', cmap='viridis', 
    #            extent=[time[0], time[-1], n_channels, 0],vmin=-1,vmax=3)  # Extent flips the y-axis to show the first channel at the top

    # # Add a vertical line at the concatenation point
    # plt.axvline(x=concat_point, color='black', linestyle='--', label='Concatenation Point')

    # # Add labels and colorbar
    # plt.colorbar(label='Z-score')
    # plt.xlabel('Time (s)')
    # plt.ylabel('Electrodes')
    # plt.title('Heatmap of Normalized Signal Amplitudes')
    # plt.yticks(ticks=np.arange(len(electrode_names)), labels=electrode_names)
    # plt.legend()
    # plt.show()

    return data_normalized_ch

file_pairs = [
    ('Sujeto6_R_processed.fif', 'Sujeto6_MF_processed.fif'),
    ('Sujeto9_R_processed.fif', 'Sujeto9_MF_processed.fif'),
    ('Sujeto11_R_processed.fif', 'Sujeto11_MF_processed.fif'),
    ('Sujeto14_R_processed.fif', 'Sujeto14_MF_processed.fif'),
    ('Sujeto16_R_processed.fif', 'Sujeto16_MF_processed.fif'),
    ('Sujeto18_R_processed.fif', 'Sujeto18_MF_processed.fif'),
    ('Sujeto19_R_processed.fif', 'Sujeto19_MF_processed.fif'),
    ('Sujeto20_R_processed.fif', 'Sujeto20_MF_processed.fif'),
    ('Sujeto21_R_processed.fif', 'Sujeto21_MF_processed.fif'),
    ('Sujeto22_R_processed.fif', 'Sujeto22_MF_processed.fif')
]

# Initialize list to store the matrices
normalized_matrices = []

# Process each pair of files
for file_R, file_MF in file_pairs:
    normalized_matrix = process_and_visualize_eeg(file_R, file_MF)
    normalized_matrices.append(normalized_matrix)

# Now calculate the average matrix
# To ensure that all matrices are the same shape, we trim them to the smallest one
min_shape = min([matrix.shape[1] for matrix in normalized_matrices])  # Smallest time dimension
normalized_matrices_trimmed = [matrix[:, :min_shape] for matrix in normalized_matrices]

# Calculate the average across all matrices
average_matrix = np.mean(np.array(normalized_matrices_trimmed), axis=0)

# Create the heatmap for the average matrix
plt.figure(figsize=(12, 6))
plt.imshow(average_matrix, aspect='auto', cmap='viridis', 
            vmin=-1, vmax=1)  # Extent flips the y-axis to show the first channel at the top
plt.colorbar(label='Z-score')
plt.xlabel('Time (s)')
plt.ylabel('Electrodes')
plt.title('Heatmap of Average Normalized Signal Amplitudes')
plt.show()



Reading c:\Users\afcad\Downloads\Sujeto6_R_processed.fif ...
    Found the data of interest:
        t =   -2500.00 ...    2500.00 ms
        0 CTF compensation matrices available


C:\Users\afcad\AppData\Local\Temp\ipykernel_24308\2736908868.py:17: RuntimeWarning: This filename (Sujeto6_R_processed.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  fileR = mne.read_epochs(file_R)


Not setting metadata
72 matching events found
No baseline correction applied
0 projection items activated
Reading c:\Users\afcad\Downloads\Sujeto6_MF_processed.fif ...


C:\Users\afcad\AppData\Local\Temp\ipykernel_24308\2736908868.py:18: RuntimeWarning: This filename (Sujeto6_MF_processed.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  fileMF = mne.read_epochs(file_MF)


    Found the data of interest:
        t =   -2500.00 ...    2500.00 ms
        0 CTF compensation matrices available
Not setting metadata
107 matching events found
No baseline correction applied
0 projection items activated
Reading c:\Users\afcad\Downloads\Sujeto9_R_processed.fif ...
    Found the data of interest:
        t =   -2500.00 ...    2500.00 ms
        0 CTF compensation matrices available


...

    Found the data of interest:
        t =   -2500.00 ...    2500.00 ms
        0 CTF compensation matrices available
Not setting metadata
106 matching events found
No baseline correction applied
0 projection items activated
Reading c:\Users\afcad\Downloads\Sujeto22_R_processed.fif ...


C:\Users\afcad\AppData\Local\Temp\ipykernel_24308\2736908868.py:17: RuntimeWarning: This filename (Sujeto22_R_processed.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  fileR = mne.read_epochs(file_R)


    Found the data of interest:
        t =   -2500.00 ...    2500.00 ms
        0 CTF compensation matrices available
Not setting metadata
67 matching events found
No baseline correction applied
0 projection items activated
Reading c:\Users\afcad\Downloads\Sujeto22_MF_processed.fif ...


C:\Users\afcad\AppData\Local\Temp\ipykernel_24308\2736908868.py:18: RuntimeWarning: This filename (Sujeto22_MF_processed.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  fileMF = mne.read_epochs(file_MF)


    Found the data of interest:
        t =   -2500.00 ...    2500.00 ms
        0 CTF compensation matrices available
Not setting metadata
138 matching events found
No baseline correction applied
0 projection items activated
